In [1]:
# DEPENDENCIES
# Python native
import functools
import json
import os
os.chdir("/home/tim/Development/OCPPM/")
import pickle
import logging
import random
from pprint import pprint
from copy import copy
from datetime import datetime
from statistics import median as median
from sys import platform
from typing import Any, Callable, Union

# Data handling
import numpy as np
import ocpa.algo.predictive_monitoring.factory as feature_factory

# PyG
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as O

# PyTorch TensorBoard support
import torch.utils.tensorboard
import torch_geometric.nn as pygnn
import torch_geometric.transforms as T

# Object centric process mining
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage

# # Simple machine learning models, procedure tools, and evaluation metrics
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import tensor
from torch.utils.tensorboard.writer import SummaryWriter
from torch_geometric.loader import DataLoader
from tqdm import tqdm

import utilities.evaluation_utils as evaluation_utils
import utilities.hetero_data_utils as hetero_data_utils
import utilities.hetero_evaluation_utils as hetero_evaluation_utils
import utilities.hetero_training_utils as hetero_training_utils
import utilities.torch_utils

# Custom imports
# from loan_application_experiment.feature_encodings.efg.efg import EFG
from torch_geometric.data import HeteroData
from experiments.hoeg import HOEG

# from importing_ocel import build_feature_storage, load_ocel, pickle_feature_storage
from models.definitions.geometric_models import GraphModel, HeteroHigherOrderGNN

# Print system info
utilities.torch_utils.print_system_info()
utilities.torch_utils.print_torch_info()

# INITIAL CONFIGURATION
cs_hoeg_config = {
    "model_output_path": "models/CS/hoeg",
    "STORAGE_PATH": "data/CS/feature_encodings/HOEG/hoeg",
    "SPLIT_FEATURE_STORAGE_FILE": "CS_split_[C2_P2_P3_O3_eas].fs",
    "OBJECTS_DATA_DICT": "cs_ofg+oi_graph+krs_node_map+krv_node_map+cv_node_map.pkl",
    "events_target_label": (feature_factory.EVENT_REMAINING_TIME, ()),
    "objects_target_label": "@@object_lifecycle_duration",
    "regression_task": True,
    "target_node_type": "event",
    "object_types": ["krs", "krv", "cv"],
    "meta_data": (
        ["event", "krs", "krv", "cv"],
        [
            ("event", "follows", "event"),
            ("event", "interacts", "krs"),
            ("event", "interacts", "krv"),
            ("event", "interacts", "cv"),
        ],
    ),
    "BATCH_SIZE": 16,
    "RANDOM_SEED": 42,
    "EPOCHS": 32,
    "early_stopping": 4,
    "optimizer_settings": {
        "lr": 0.001,
        "betas": (0.9, 0.999),
        "eps": 1e-08,
        "weight_decay": 0,
        "amsgrad": False,
    },
    "loss_fn": torch.nn.L1Loss(),
    "verbose": True,
    "skip_cache": False,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "squeeze_required": False,    
}

# CONFIGURATION ADAPTATIONS may be set here
# cs_hoeg_config["early_stopping"] = 4
# cs_hoeg_config['skip_cache'] = True
cs_hoeg_config['device'] = torch.device('cpu')
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    filename="logging/debug.log",
)
logging.critical("-" * 32 + ' TEST CS HOEG ' + "-" * 32)

CRITICAL:root:-------------------------------- TEST CS HOEG --------------------------------
CRITICAL:root:-------------------------------- TEST CS HOEG --------------------------------


CPU: Intel(R) Core(TM) i5-7500 CPU @ 3.40GHz (4x)
Total CPU memory: 46.93GB
Available CPU memory: 37.11GB
GPU: NVIDIA GeForce GTX 960
Total GPU memory: 4096.0MB
Available GPU memory: 4029.0MB
Platform: Linux-5.19.0-46-generic-x86_64-with-glibc2.35
Torch version: 1.13.1+cu117
Cuda available: True
Torch geometric version: 2.3.1


In [2]:
def evaluate_hetero_model(
    target_node_type: str,
    model: GraphModel,
    dataloader: DataLoader,
    metric: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    device: torch.device = torch.device("cpu"),
    verbose: bool = False,
    squeeze_required: bool = True,
) -> torch.Tensor:
    with torch.no_grad():

        def _eval_batch(batch, model):
            batch_inputs, batch_adjacency_matrix, batch_labels = (
                batch.x_dict,
                batch.edge_index_dict,
                batch[target_node_type].y,
            )
            return (
                model(
                    batch_inputs,
                    edge_index=batch_adjacency_matrix
                    # , batch=batch[target_node_type].batch,
                ),
                batch_labels,
            )

        model.eval()
        model.train(False)
        model.to(device)
        y_preds = torch.tensor([]).to(device)
        y_true = torch.tensor([]).to(device)
        for batch in tqdm(dataloader, disable=not (verbose)):
            batch.to(device)
            batch_y_preds, batch_y_true = _eval_batch(batch, model)
            # append
            y_preds = torch.cat(
                (y_preds, batch_y_preds[target_node_type].view(-1, 25).mean(dim=1))
            )
            y_true = torch.cat((y_true, batch_y_true))
        if squeeze_required:
            y_preds = torch.squeeze(y_preds)
    return metric(y_preds.to(device), y_true.to(device))


def evaluate_best_model(
    target_node_type: str,
    model_state_dict_path: str,
    model: GraphModel,
    metric: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    device: torch.device,
    train_loader: Union[DataLoader, None] = None,
    val_loader: Union[DataLoader, None] = None,
    test_loader: Union[DataLoader, None] = None,
    verbose: bool = True,
    squeeze_required: bool = True,
) -> dict[str, torch.Tensor]:
    best_state_dict = torch.load(model_state_dict_path, map_location=device)

    model.load_state_dict(best_state_dict)
    model.eval()
    kwargs = {
        "target_node_type": target_node_type,
        "model": model,
        "metric": metric,
        "device": device,
        "verbose": verbose,
        "squeeze_required": squeeze_required,
    }
    evaluation = {}
    if train_loader:
        evaluation |= {
            f"Train {metric}": evaluate_hetero_model(dataloader=train_loader, **kwargs)
        }
    if val_loader:
        evaluation |= {
            f"Val {metric}": evaluate_hetero_model(dataloader=val_loader, **kwargs)
        }
    if test_loader:
        evaluation |= {
            f"Test {metric}": evaluate_hetero_model(dataloader=test_loader, **kwargs)
        }
    return evaluation


In [3]:
# DATA PREPARATION
transformations = [
    T.ToUndirected(),  # Convert the graph to an undirected graph
    # T.AddSelfLoops(),  # Add self-loops to the graph
    # T.NormalizeFeatures(),  # Normalize node features of the graph
]
# Get data and dataloaders
ds_train, ds_val, ds_test = hetero_data_utils.load_hetero_datasets(
    storage_path=cs_hoeg_config["STORAGE_PATH"],
    split_feature_storage_file=cs_hoeg_config["SPLIT_FEATURE_STORAGE_FILE"],
    objects_data_file=cs_hoeg_config["OBJECTS_DATA_DICT"],
    event_node_label_key=cs_hoeg_config["events_target_label"],
    object_nodes_label_key=cs_hoeg_config['objects_target_label'],
    edge_types=cs_hoeg_config['meta_data'][1],
    object_node_types=cs_hoeg_config['object_types'],
    graph_level_target=False,
    transform=T.Compose(transformations),
    train=True,
    val=True,
    test=True,
    skip_cache=cs_hoeg_config["skip_cache"],
)


In [4]:
# update meta data
cs_hoeg_config["meta_data"] = ds_val[0].metadata()
# print_hetero_dataset_summaries(ds_train, ds_val, ds_test)
(
    train_loader,
    val_loader,
    test_loader,
) = hetero_data_utils.hetero_dataloaders_from_datasets(
    batch_size=cs_hoeg_config["BATCH_SIZE"],
    ds_train=ds_train,
    ds_val=ds_val,
    ds_test=ds_test,
    seed_worker=functools.partial(
        utilities.torch_utils.seed_worker, state=cs_hoeg_config["RANDOM_SEED"]
    ),
    generator=torch.Generator().manual_seed(cs_hoeg_config["RANDOM_SEED"]),
)

In [5]:
# MODEL INITIATION
# TODO: try custom Heterogeneous GNN Architecture (without to_hetero())
class HeteroHigherOrderGNN(GraphModel):
    def __init__(
        self,
        hidden_channels: int = 32,
        out_channels: int = 1,
        regression_target: bool = True,
    ):
        super().__init__()
        self.conv1 = pygnn.GraphConv(-1, hidden_channels)
        self.act1 = nn.PReLU()
        # self.conv2 = pygnn.GraphConv(-1, hidden_channels, add_self_loops=False)
        # self.act2 = nn.PReLU()
        self.lin_out = pygnn.Linear(-1, out_channels)
        # self.probs_out = lambda x: x
        # if not regression_target:
        #     self.probs_out = nn.Softmax(dim=1)

    def forward(self, x, edge_index, batch=None):
        x = x.view(-1,1)
        x = self.conv1(x, edge_index)
        x = self.act1(x)
        # x = self.conv2(x, edge_index)
        # x = self.act2(x)
        x = self.lin_out(x)
        # return self.probs_out(x)
        return x

model = HeteroHigherOrderGNN(32, 1, cs_hoeg_config['regression_task'])
model = pygnn.to_hetero(model, cs_hoeg_config["meta_data"])

# Print summary of data and model
cs_hoeg_config["verbose"] = True
if cs_hoeg_config["verbose"]:
    # print(model)
    with torch.no_grad():  # Initialize lazy modules, s.t. we can count its parameters.
        batch = next(iter(train_loader))
        batch.to(cs_hoeg_config["device"])
        model.to(cs_hoeg_config["device"])
        out = model(batch.x_dict, batch.edge_index_dict)
        print(f"Number of parameters: {utilities.torch_utils.count_parameters(model)}")

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/nn/to_hetero_transformer.py:379: UserWarning: 'act1' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'act1'
  warnings.warn(


/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [1], which does not match the required output shape [1, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [6], which does not match the required output shape [6, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, i

Number of parameters: 808


In [6]:
# MODEL TRAINING
print("Training started, progress available in Tensorboard")
torch.cuda.empty_cache()

timestamp = datetime.now().strftime("%Y%m%d_%Hh%Mm")
model_path_base = f"{cs_hoeg_config['model_output_path']}/{str(model).split('(')[0]}_{timestamp}"

best_state_dict_path = hetero_training_utils.run_training_hetero(
    target_node_type=cs_hoeg_config["target_node_type"],
    num_epochs=cs_hoeg_config["EPOCHS"],
    model=model,
    train_loader=train_loader,
    validation_loader=val_loader,
    optimizer=O.Adam(model.parameters(), **cs_hoeg_config["optimizer_settings"]),
    loss_fn=cs_hoeg_config["loss_fn"],
    early_stopping_criterion=cs_hoeg_config["early_stopping"],
    model_path_base=model_path_base,
    device=cs_hoeg_config["device"],
    verbose=False,
    squeeze_required=cs_hoeg_config['squeeze_required']
)

# Write experiment settings as JSON into model path (of the model we've just trained)
with open(os.path.join(model_path_base, "experiment_settings.json"), "w") as file_path:
    json.dump(evaluation_utils.get_json_serializable_dict(cs_hoeg_config), file_path)

Training started, progress available in Tensorboard


  0%|          | 0/5058 [00:00<?, ?it/s]

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [2], which does not match the required output shape [2, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [11], which does not match the required output shape [11, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it,

Early stopping after 9 epochs.


In [11]:
# MODEL EVALUATION
model_path_base = f"{cs_hoeg_config['model_output_path']}/GraphModule_20230802_13h34m"
state_dict_path = f"{cs_hoeg_config['model_output_path']}/GraphModule_20230802_13h34m/state_dict_epoch2.pt"  # 0.5517 test mae | 15k params
# model_path_base = f"{cs_hoeg_config['model_output_path']}/GraphModule_20230802_16h49m"
# state_dict_path = f"{cs_hoeg_config['model_output_path']}/GraphModule_20230802_16h49m/state_dict_epoch4.pt"  # 0.5514 test mae | 808 params

# Get evaluation results
# evaluation_dict = hetero_evaluation_utils.evaluate_best_model(
evaluation_dict = evaluate_best_model(
    target_node_type=cs_hoeg_config["target_node_type"],
    model_state_dict_path=state_dict_path,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    model=model,
    metric=cs_hoeg_config["loss_fn"],
    device=cs_hoeg_config["device"],
    verbose=cs_hoeg_config["verbose"],
    squeeze_required=cs_hoeg_config['squeeze_required']
)

# Store model results as JSON into model path
with open(os.path.join(model_path_base, "evaluation_report.json"), "w") as file_path:
    json.dump(evaluation_utils.get_json_serializable_dict(evaluation_dict), file_path)

  0%|          | 0/5058 [00:00<?, ?it/s]

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [14], which does not match the required output shape [14, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [6], which does not match the required output shape [6, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it,

In [13]:
# Print MAE results
print(model_path_base)
pprint(evaluation_dict)

models/CS/hoeg/GraphModule_20230802_13h34m
{'Test L1Loss()': tensor(0.5517),
 'Train L1Loss()': tensor(0.5572),
 'Val L1Loss()': tensor(0.5563)}
